In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/LTC-USD.csv
/kaggle/input/BCH-USD.csv
/kaggle/input/BTC-USD.csv
/kaggle/input/ETH-USD.csv


In [2]:
data1 = pd.read_csv('/kaggle/input/LTC-USD.csv', names = ["time","LTC-low","LTC-high","LTC-open","LTC-close","LTC-volume"])


data2 = pd.read_csv('/kaggle/input/BTC-USD.csv', names = ["time","BTC-low","BTC-high","BTC-open","BTC-close","BTC-volume"])


data3= pd.read_csv('/kaggle/input/ETH-USD.csv', names = ["time","ETH-low","ETH-high","ETH-open","ETH-close","ETH-volume"])


data4 = pd.read_csv('/kaggle/input/BCH-USD.csv', names = ["time","BCH-low","BCH-high","BCH-open","BCH-close","BCH-volume"])

data2.head()

,time,BTC-low,BTC-high,BTC-open,BTC-close,BTC-volume
0,1528968660,6489.549805,6489.560059,6489.560059,6489.549805,0.587100
1,1528968720,6487.370117,6489.560059,6489.549805,6487.379883,7.706374
2,1528968780,6479.410156,6487.370117,6487.370117,6479.410156,3.088252
3,1528968840,6479.410156,6479.419922,6479.419922,6479.410156,1.404100
4,1528968900,6475.930176,6479.979980,6479.410156,6479.979980,0.753000


In [3]:
main_df = pd.DataFrame()

In [4]:
ratios = ["BTC-USD", "LTC-USD", "BCH-USD", "ETH-USD"]  # the 4 ratios we want to consider
for ratio in ratios:  # begin iteration
    print(ratio)
    dataset = f'/kaggle/input/{ratio}.csv'  # get the full path to the file.
    df = pd.read_csv(dataset, names=['time', 'low', 'high', 'open', 'close', 'volume'])  # read in specific file

    # rename volume and close to include the ticker so we can still which close/volume is which:
    df.rename(columns={"close": f"{ratio}_close", "volume": f"{ratio}_volume"}, inplace=True)

    df.set_index("time", inplace=True)  # set time as index so we can join them on this shared time
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]  # ignore the other columns besides price and volume

    if len(main_df)==0:  # if the dataframe is empty
        main_df = df  # then it's just the current df
    else:  # otherwise, join this data to the main one
        main_df = main_df.join(df)

main_df.fillna(method="ffill", inplace=True)  # if there are gaps in data, use previously known values
main_df.dropna(inplace=True)
print(main_df.head())  # how did we do??

BTC-USD
LTC-USD
BCH-USD
ETH-USD
            BTC-USD_close  BTC-USD_volume  LTC-USD_close  LTC-USD_volume  \
time                                                                       
1528968720    6487.379883        7.706374      96.660004      314.387024   
1528968780    6479.410156        3.088252      96.570000       77.129799   
1528968840    6479.410156        1.404100      96.500000        7.216067   
1528968900    6479.979980        0.753000      96.389999      524.539978   
1528968960    6480.000000        1.490900      96.519997       16.991997   

            BCH-USD_close  BCH-USD_volume  ETH-USD_close  ETH-USD_volume  
time                                                                      
1528968720     870.859985       26.856577      486.01001       26.019083  
1528968780     870.099976        1.124300      486.00000        8.449400  
1528968840     870.789978        1.749862      485.75000       26.994646  
1528968900     870.000000        1.680500      486.00000    

In [5]:
SEQ_LEN = 60  # how long of a preceeding sequence to collect for RNN
FUTURE_PERIOD_PREDICT = 3  # how far into the future are we trying to predict?
RATIO_TO_PREDICT = "LTC-USD"

def classify(current, future):
    if float(future) > float(current):
        return 1
    else:
        return 0
    
main_df['future'] = main_df[f'{RATIO_TO_PREDICT}_close'].shift(-FUTURE_PERIOD_PREDICT)

main_df['target'] = list(map(classify, main_df[f'{RATIO_TO_PREDICT}_close'], main_df['future']))

#print(main_df[[f"{RATIO_TO_PREDICT}_close","future"]])

main_df['target'] = list(map(classify,main_df[f"{RATIO_TO_PREDICT}_close"],main_df['future']))

print(main_df[[f"{RATIO_TO_PREDICT}_close","future","target"]])

main_df.dropna(inplace = True)


            LTC-USD_close     future  target
time                                        
1528968720      96.660004  96.389999       0
1528968780      96.570000  96.519997       0
1528968840      96.500000  96.440002       0
1528968900      96.389999  96.470001       1
1528968960      96.519997  96.400002       0
...                   ...        ...     ...
1535214960      58.020000  58.020000       0
1535215020      58.009998  58.080002       1
1535215080      58.020000        NaN       0
1535215140      58.020000        NaN       0
1535215200      58.080002        NaN       0

[97723 rows x 3 columns]


In [6]:
print(main_df[[f"{RATIO_TO_PREDICT}_close","future","target"]])


            LTC-USD_close     future  target
time                                        
1528968720      96.660004  96.389999       0
1528968780      96.570000  96.519997       0
1528968840      96.500000  96.440002       0
1528968900      96.389999  96.470001       1
1528968960      96.519997  96.400002       0
...                   ...        ...     ...
1535214780      58.009998  58.020000       1
1535214840      58.009998  58.009998       0
1535214900      58.020000  58.020000       0
1535214960      58.020000  58.020000       0
1535215020      58.009998  58.080002       1

[97720 rows x 3 columns]


In [7]:
times = sorted(main_df.index.values)

last_5pct = times[-int(0.5*len(times))]
print(last_5pct)

1531906380


In [8]:
validation_main_df =  main_df[(main_df.index)>=last_5pct]
main_df =  main_df[(main_df.index)<last_5pct]


In [9]:
from sklearn import preprocessing
from collections import deque
import random

def preprocess_df(df):
    df = df.drop('future',axis = 1)
    
    for col in df.columns:
        if col != "target":
            df[col] =df[col].pct_change()
            df = df.dropna()
            df[col] = preprocessing.scale(df[col].values)
            
    df.dropna(inplace = True)
    
    sequential_data =[]
    prev_days = deque(maxlen = SEQ_LEN)
    
    for i in df.values:
        prev_days.append([n for n in i[:-1]])
        
        if(len(prev_days)==SEQ_LEN):
            sequential_data.append([np.array(prev_days), i[-1]])
            
    random.shuffle(sequential_data)
    
    buys = []
    sell = []

    for seq, target in sequential_data:
        if target ==0:
            sell.append([seq,target])
        else:
            buys.append([seq,target])

    random.shuffle(buys)
    random.shuffle(sell)

    lower = min(len(buys),len(sell))

    buys = buys[:lower]
    sell = sell[:lower]

    sequential_data = buys+sell

    random.shuffle(sequential_data)

    X= []
    y= []

    for seq,target in sequential_data:
        X.append(seq)
        y.append(target)
        
    return np.array(X),np.array(y)
    


train_x,train_y = preprocess_df(main_df)
validation_x,validation_y = preprocess_df(validation_main_df)



In [10]:
print(f"train data: {len(train_x)} validation: {len(validation_x)}")
#print(f"Dont buys: {train_y.count(0)}, buys: {train_y.count(1)}")
#print(f"VALIDATION Dont buys: {validation_y.count(0)}, buys: {validation_y.count(1)}")

train data: 41084 validation: 40670


In [11]:
import time
EPOCHS = 10
BATCH_SIZE = 64
NAME = f"{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{int(time.time())}"  # a unique name for the model


In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard,ModelCheckpoint

model = Sequential()
model.add(LSTM(128, input_shape=(train_x.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())  #normalizes activation outputs, same reason you want to normalize your input data.

model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax'))

import tensorflow as tf
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

# Compile model
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)
tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

filepath = "RNN_Final-{epoch:02d}-{val_acc:.3f}"  # unique file name that will include the epoch and the validation acc for that epoch
#checkpoint = ModelCheckpoint("models/{}.model".format(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')) # saves only the best ones
history = model.fit(
    train_x, train_y,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(validation_x, validation_y),
)
score = model.evaluate(validation_x, validation_y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
# Save model
model.save("models/{}".format(NAME))

Epoch 1/10
642/642 [==============================] - 14s 22ms/step - loss: 0.7326 - accuracy: 0.5143 - val_loss: 0.6981 - val_accuracy: 0.5129
Epoch 2/10
642/642 [==============================] - 13s 20ms/step - loss: 0.6928 - accuracy: 0.5338 - val_loss: 0.6901 - val_accuracy: 0.5298
Epoch 3/10
642/642 [==============================] - 13s 20ms/step - loss: 0.6877 - accuracy: 0.5417 - val_loss: 0.6899 - val_accuracy: 0.5364
Epoch 4/10
642/642 [==============================] - 14s 21ms/step - loss: 0.6853 - accuracy: 0.5532 - val_loss: 0.6850 - val_accuracy: 0.5512
Epoch 5/10
642/642 [==============================] - 13s 20ms/step - loss: 0.6806 - accuracy: 0.5618 - val_loss: 0.6850 - val_accuracy: 0.5566
Epoch 6/10
642/642 [==============================] - 13s 20ms/step - loss: 0.6772 - accuracy: 0.5710 - val_loss: 0.6900 - val_accuracy: 0.5503
Epoch 7/10
642/642 [==============================] - 13s 21ms/step - loss: 0.6747 - accuracy: 0.5744 - val_loss: 0.6847 - val_accuracy: